In [2]:
import numpy as np

import qutip
from qutip import *

import scipy.interpolate

In [3]:
PI_2 = 2 * np.pi

Nosc = 1 # number of coupled oscillators
Nctrl = 1
Ncoupled = 1

Ne1 = 2 # essential energy levels per oscillator
Ng1 = 0 # Osc-1, number of guard states

Ne = [Ne1]
Ng = [Ng1]

N = Ne1; # Total number of nonpenalized energy levels
Ntot = (Ne1+Ng1)

Nguard = Ntot - N # total number of guard states

Nt1 = Ne1 + Ng1

alpha = 200e-3

ALPHA = PI_2 * (-1) * alpha

ALPHA_BY_2 = ALPHA/2

In [4]:
# qubit:
B = destroy(Nt1)

H_0 = ALPHA_BY_2*B.dag()*B.dag()*B*B

H_c = [1*(B+B.dag()),-1j*(B-B.dag())]

In [5]:
H_0, H_c

(Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[0. 0.]
  [0. 0.]],
 [Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0. 1.]
   [1. 0.]],
  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0.+0.j 0.-1.j]
   [0.+1.j 0.+0.j]]])

In [6]:
def time_evolution_with_psi0(t, psi0, final_amps):
    # operators and the hamiltonian
    sx = qutip.sigmax()
    sy = qutip.sigmay()
    sz = qutip.sigmaz()
    sm = qutip.sigmam()

    drive_temp = np.empty([len(H_c), len(final_amps[0])])
    for k in range(len(H_c)):
        drive_temp[k] = final_amps[k].flatten()
    drive_temp = drive_temp.T[:,:]
    print(drive_temp.shape)

    print("H_c has ",len(H_c)," terms")
    print("drive data has ",len(drive_temp[0,:])," terms")

    #Helper function
    def mkinterp(x,y,kind):
        lx = x
        ly = y
        lkind = kind
        func = scipy.interpolate.interp1d(lx,ly,kind=lkind,bounds_error=False,fill_value=0.0)
        return func

    #Helper function
    #combine time independent Hamiltonian with control terms from H_c
    #and drive functions loaded from driveFile to define time dependent hamiltonian
    def mk_H_t(driveFile,H_0,H_c,tvec_in,intrp_type='zero'):
        iamps = [] #array storing interpolation functions for each drive
        H = [H_0]
        n_ctrls = len(H_c)
        funcs = []
        for ctrl_idx in range(n_ctrls):
            #interpolate tabular data in drive_data
            funcs.append(mkinterp(tvec_in,driveFile[:,ctrl_idx],intrp_type))
            H.append([H_c[ctrl_idx],lambda t,args,idx=ctrl_idx: funcs[idx](t)])
        return H

    # %%time
    #Essentially use this function to combine the time indpedent and time dependent Hamiltonian into one object
    H_sys_qsps_ex = mk_H_t(drive_temp,H_0,H_c,t)

    options = Options(atol=1e-8,rtol=1e-6,nsteps=1000)

    # Use this line below to get gates
    driven_sys_qsps_ex = propagator(H_sys_qsps_ex, t, [], options=options) #, parallel = True, num_cpus = 8) # , unitary_mode = 'single') #, parallel = True)
    return driven_sys_qsps_ex[-1] # Modified to give gate instead

In [15]:
def gate_overlap(pred, golden):
    assert(pred.shape == golden.shape and pred.shape[0] == pred.shape[1])
    # Eq. 7: https://arxiv.org/pdf/2110.12570.pdf
    return ((pred.trans().conj()*pred).tr()+(abs((golden.trans().conj()*pred).tr())**2))\
                /(pred.shape[0]*(pred.shape[0]+1))

def process_qmodel1_file(set_num):
    # qubit:
    B = destroy(Nt1)

    ##########

    # pfunc
    data_array_ctrl_rot1 = np.loadtxt('../juqbox_env/model-processed-' + str(set_num) + '-ctrl-rot-1.dat')
    data_array_ctrl_rot1_actual = np.loadtxt('../juqbox_env/model-processed-' + str(set_num+200) + '-ctrl-rot-1.dat')
    with open("../scratch/hls-outputs/tfout_" + str(set_num) + ".txt", "r") as f:
        data_angle = float(f.readlines()[20])
        print("Data Angle: ", data_angle)

    final_amps = ["str%d" % x for x in range(Nosc * 2)]

    final_amps[0] = data_array_ctrl_rot1[0]
    final_amps[1] = data_array_ctrl_rot1[1]

    final_amps_actual = ["str%d" % x for x in range(Nosc * 2)]
    final_amps_actual[0] = data_array_ctrl_rot1_actual[0]
    final_amps_actual[1] = data_array_ctrl_rot1_actual[1]

    ##########
    
    Tmax = 100
    samplerate = 32

    t = np.linspace(0,Tmax,Tmax*samplerate + 1)

    # initial state
    a = 1.0
    psi0 = (a*qutip.basis(2, 0) + (1-a)*qutip.basis(2, 1))/np.sqrt(a**2 + (1-a)**2)

    gate_pred = time_evolution_with_psi0(t, psi0, final_amps)
    gate_actual = time_evolution_with_psi0(t, psi0, final_amps_actual)

    ##########

    sigma_x = np.array([[0,1],\
            [1,0]])

    U0 = qutip.qobj.Qobj(np.array([[1,0],[0,1]]))

    gate_golden = qutip.qobj.Qobj(-1j*data_angle/2*sigma_x).expm()

    gate_golden = U0 * gate_golden

    ##########

    overlap_po = gate_overlap(gate_pred, gate_actual)
    overlap_pg = gate_overlap(gate_pred, gate_golden)
    overlap_ag = gate_overlap(gate_actual, gate_golden)

    print("Overlap (predicted and optimized): ", overlap_po)
    print("Overlap (predicted and golden): ", overlap_pg)
    print("Overlap (optimized and golden): ", overlap_ag)

    with open("../scratch/fid-outputs/fidout_" + str(set_num) + ".txt", "w") as f:
        f.write(str(set_num) + "," + str(data_angle) + "," + str(overlap_po) + "," + str(overlap_pg) + "," + str(overlap_ag) + "\n")
        print("Wrote data for: ", set_num, data_angle)


In [ ]:
# For each beta, calculate and write gate fidelities
for i in range(100):
    process_qmodel1_file(i)